In [11]:
import openai
import pinecone
import os
import huggingface_hub
import pinecone
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
huggingface_api_token = os.getenv('HUGGINGFACE_API_TOKEN')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
openai.api_key = openai_api_key
huggingface_hub.api_token = huggingface_api_token
pinecone.api_key = pinecone_api_key
import openai
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [12]:
def load_docs(directory):
  loader = PyPDFDirectoryLoader(directory)
  documents = loader.load()
  return documents
directory = '.'
documents = load_docs(directory)
len(documents)
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs
docs = split_docs(documents)

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
query_result = embeddings.embed_query("Hello Buddy")
len(query_result)
import pinecone
pinecone.init(
    api_key='',
    environment="gcp-starter"
)

index_name = "mcq-creator"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)
def get_similiar_docs(query, k=2):
    similar_docs = index.similarity_search(query, k=k)
    return similar_docs
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
llm=HuggingFaceHub(repo_id="bigscience/bloom", model_kwargs={"temperature":1e-10})
llm
chain = load_qa_chain(llm, chain_type="stuff")
def get_answer(query):
  relevant_docs = get_similiar_docs(query)
  print(relevant_docs)
def get_answer(query):
  relevant_docs = get_similiar_docs(query)
  print(relevant_docs)
  response = chain.run(input_documents=relevant_docs, question=query)
  return response
answer= get_answer("How is India's economy?")
import re
import json

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
response_schemas = [
    ResponseSchema(name="question", description="Question generated from provided input text data."),
    ResponseSchema(name="choices", description="Available options for a multiple-choice question in comma separated."),
    ResponseSchema(name="answer", description="Correct answer for the asked question.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
output_parser
# This helps us fetch the instructions the langchain creates to fetch the response in desired format
format_instructions = output_parser.get_format_instructions()
 
print(format_instructions)
# create ChatGPT object
chat_model = ChatOpenAI()

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("""When a text input is given by the user, please generate multiple choice questions 
        from it along with the correct answer. 
        \n{format_instructions}\n{user_prompt}""")  
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)
final_query = prompt.format_prompt(user_prompt = answer)
print(final_query)
final_query.to_messages()
final_query_output = chat_model(final_query.to_messages())
print(final_query_output.content)

# Let's extract JSON data from Markdown text that we have
markdown_text = final_query_output.content
json_string = re.search(r'{(.*?)}', markdown_text, re.DOTALL).group(1)

TypeError: expected string or bytes-like object

In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [14]:
llm = OpenAI()

In [15]:
prompt = PromptTemplate(
    input_variables=["user_prompt"],
    template ="Best places to visit in {user_prompt}",
)

In [16]:
chain = LLMChain(llm=llm, prompt=prompt)

In [17]:
chain.run(user_prompt="Singapore")   

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-pX16n***************************************h2kj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}